# Tunnel Transit Time Calculator
## Overview
As stated in the [Transport Manifesto](https://github.com/dsposito/transport-manifesto), [tunnels](https://github.com/dsposito/transport-manifesto/tree/master/tunnels) are a mode of transport with a number of benefits. For example, tunnels can provide dramatically faster travel from point A to B due to 1.) its use of autonomous electric vehicles and 2.) support for higher speeds (155mph+).

**The purpose of this interactive notebook is to:**
1. Calculate the estimated time savings of traveling by tunnels vs traditional roadways
2. Easily inspect, edit and evolve the underlying logic that drives the comparison calculations

## Demo

> **Try the interactive calculator: [https://tunnel-transit-time-calculator.herokuapp.com](https://tunnel-transit-time-calculator.herokuapp.com)**

_Continue reading below for a walkthrough of how the comparison metrics are calculated._

## Installation
Follow these instructions if you want to run and edit this notebook locally:

> This notebook uses the Google Maps API. Create your key using Google's [Developer Console](https://console.developers.google.com/flows/enableapi?apiid=maps_backend,geocoding_backend,directions_backend,distance_matrix_backend,elevation_backend&keyType=CLIENT_SIDE&reusekey=true).

0. Add your Google Maps API key as the value for `GMAPS_API_KEY` in the `tunnels/.env` file
1. From the repo's root directory, install the dependencies: `pip install -r requirements.txt`
2. Navigate to the `transit-time-calculator` directory
3. Enable the [ipywidgets](https://ipywidgets.readthedocs.io) extension: `jupyter nbextension enable --py --sys-prefix widgetsnbextension`
4. Enable the [gmaps](https://jupyter-gmaps.readthedocs.io) widget extension: `jupyter nbextension enable --py --sys-prefix gmaps`
5. Run the notebook: `jupyter notebook`

## Code Walkthrough

At a high level, there are three major code components:
1. Displaying a form to input an origin and destination address
2. Displaying the route via Google Maps + determining the total distance to travel
3. Displaying comparison metrics for traveling by roadways vs tunnels

### Imports
Import the relevant modules and packages.

In [1]:
import math
import os

from dotenv import load_dotenv, find_dotenv
import gmaps as gmaps_ui
import googlemaps as gmaps_api
import ipywidgets as widgets
from IPython.display import display, Markdown as md

### APIs
Init the Google Maps widget and backend API clients with credentials.

In [2]:
load_dotenv(find_dotenv())

gmaps_ui.configure(api_key=os.getenv("GMAPS_API_KEY"))
gmaps_api = gmaps_api.Client(key=os.getenv("GMAPS_API_KEY"))

### Form Input UI
This notebook uses [ipywidgets](https://ipywidgets.readthedocs.io) to render special interactive widgets like Google Maps.

**They don't render natively on GitHub but will load locally or via binder.**

Display text inputs for origin and destination addresses. Also register the function that gets called when the "Calculate Route" button is clicked.

In [9]:
def handle_form_submit(element):
    """Handles the click even for the submit button.
    """
    
    # Clear any existing output (e.g., from previous form submit).
    form_output.clear_output()
    
    with form_output:
        # Validate the form inputs.
        if not input_origin.value or not input_destination.value:
            print("Please enter a valid address and try again.")
            return

        # Geocode the string addresses to coordinates (necessary for the gmaps widget).
        origin = geocode_address(input_origin.value)
        destination = geocode_address(input_destination.value)

        if not origin or not destination:
            print("Please enter a valid address and try again.")
            return

        # Calculate road comparison metrics (e.g., distance and time) by creating driving directions via Google Maps API.
        metrics_road = calculate_road_metrics(origin, destination)
        if not metrics_road:
            print("Unable to calculate road metrics.")
            return

        # Calculate tunnel comparison metrics using the total distance queried from Google Maps API.
        metrics_tunnel = calculate_tunnel_metrics(metrics_road["distance"]["value"])

        # Display the route in a Google Map and a table of comparison metrics.
        display_map(origin, destination)
        display_metrics(metrics_road, metrics_tunnel)

# Create the form fields.
input_origin = widgets.Text(description="Origin: ", value="Santa Monica, CA", layout={'width': '50%'})
input_destination = widgets.Text(description="Destination: ", value="Downtown Los Angeles", layout={'width': '50%'})
button_submit = widgets.Button(description="Calculate Route", button_style="Success")
button_submit.on_click(handle_form_submit)

# Display the form.
display(md("### Transit Time Comparison Calculator"))
display(md("**Give it a try!**"))
form_output = widgets.Output()

display(input_origin, input_destination, button_submit, form_output)

### Transit Time Comparison Calculator

**Give it a try!**

Text(value='Santa Monica, CA', description='Origin: ', layout=Layout(width='50%'))

Text(value='Downtown Los Angeles', description='Destination: ', layout=Layout(width='50%'))

Button(button_style='success', description='Calculate Route', style=ButtonStyle())

Output()

### Geocoding
Define a simple helper method to geocode a string address into its latitude/longitude coordinates. This is required for the gmaps widget.

In [4]:
def geocode_address(address: str) -> tuple:
    """Geocodes a string address into latitude/longitude coordinates.
    """
    
    response = gmaps_api.geocode(address)
    if not response:
        print("Unable to geocode address.")
        return

    location = response[0]["geometry"]["location"]

    return location["lat"], location["lng"]

### Core Logic: Calculating Roadway Metrics
To avoid the complexity of land ownership rights, this model assumes tunnels are underground along the path of public right-of-way highways and city streets.

Therefore, the Google Maps API is used to determine the route for BOTH traditional roadway and tunnel transit (the latter would be underground of course). This is an oversimplification given that high-speed tunnels will need to avoid sharp turns. However, it's directionally accurate and a future iteration of this notebook could create a more realistic path.

This method determines the total distance and time via Google Maps API and calculates other metrics such as speed.

In [5]:
def calculate_road_metrics(origin: tuple, destination: tuple) -> dict:
    """Calculates metrics for traveling a route via traditional roadways.
    
    The Google Maps API is queried to determine the total distance from origin to destination.
    """
    
    METERS_PER_FOOT = 0.3048
    FEET_PER_MILE = 5280
    
    CITY_TOP_SPEED = 40
    HIGHWAY_TOP_SPEED = 65
    
    # Query Google Maps API for directions data - which includes total distance and time.
    directions = gmaps_api.directions(origin, destination)
    if not directions:
        print("Unable to retrieve directions.")
        return
    
    # Convert distance from meters to miles.
    distance_miles = int(directions[0]["legs"][0]["distance"]["value"] / METERS_PER_FOOT / FEET_PER_MILE)
    
    # Convert time from seconds to minutes.
    time_minutes = directions[0]["legs"][0]["duration"]["value"] / 60
    
    # Calculate top speed by checking whether highway is used for any steps of the route.
    speed_max = CITY_TOP_SPEED
    for steps in directions[0]["legs"][0]["steps"]:
        if "ramp" in steps.get("maneuver", ""):
            speed_max = HIGHWAY_TOP_SPEED
            break
    
    # Calculate average speed: d = r*t
    time_hours = time_minutes / 60
    speed_avg = distance_miles / time_hours
    
    return {
        "distance": {
            "unit": "mi",
            "value": round(distance_miles, 1),
            "raw": distance_miles
        },
        "time": {
            "unit": "min",
            "value": round(time_minutes),
            "raw": time_minutes
        },
        "speed_max": {
            "unit": "mph",
            "value": round(speed_max),
            "raw": speed_max
        },
        "speed_avg": {
            "unit": "mph",
            "value": round(speed_avg),
            "raw": speed_avg
        }
    }

### Core Logic: Simulating Tunnel Vehicle Kinematics
Using the trip distance from the Google Maps route, we calculate the total time to travel the entire distance based on a few assumptions such as rate of acceleration and max speed allowed within the tunnel.

This assumes the travel time is composed of three elements:
1. **Departure:** Time accelerating from 0mph up to top speed
2. **Cruising:** Time spent at top speed
3. **Arrival:** Time decelerating to 0mph

The [**kinematic displacement formula**](https://www.khanacademy.org/science/physics/one-dimensional-motion/kinematic-formulas/a/what-are-the-kinematic-formulas) is used to calculate the total distance traveled during acceleration / deceleration (which is then used to calculate total travel time for the three elements above).

\begin{equation*}
\Delta x = \Bigl(\frac{v + v_0}{2}\Bigr) t 
\end{equation*}

The initial distance traveled is recalculated during *each second of acceleration* to determine the max vehicle velocity / speed reached - assuming the same amount of time is needed to fully decelerate to a stop when arriving at the destination.

> For example, it takes approximately 1 mile to accelerate to 155mph in 45 seconds. Therefore, if the total trip distance is only **1 mile** then the vehicle will only be able to reach a max speed of **~110mph** by the time it reaches halfway (0.5 miles) at which point it will need to begin decelerating.

In [6]:
def simulate_tunnel_vehicle_kinematics(distance_trip: float) -> dict:
    """Simulates the acceleration of an AEV in a tunnel to determine the top speed and total time of travel.
    """
    
    # Assumptions for Tesla Model X vehicle.
    VEHICLE_TOP_SPEED = 155 # mph
    VEHICLE_TOP_SPEED_TIME = 45 # seconds (~0.157 g-force: http://www.procato.com/convert)

    # Assume constant rate of acceleration/deceleration.
    VEHICLE_DELTA_SPEED_TIME = VEHICLE_TOP_SPEED / VEHICLE_TOP_SPEED_TIME

    speed_current = 0
    for time_elapsed in range(0, VEHICLE_TOP_SPEED_TIME + 1):
        # Calculate distance traveled up to this point using the kinematic displacement formula: d = 0.5 * v * t
        velocity_current = speed_current / (60 * 60)
        distance_current = 0.5 * velocity_current * time_elapsed

        velocity_next = (speed_current + VEHICLE_DELTA_SPEED_TIME) / (60 * 60)
        distance_next = 0.5 * velocity_next * (time_elapsed + 1)

        # Cap the velocity if the upcoming distance traveled would be more than half
        # of the total trip distance (outcome would be to begin decelerating).
        if distance_next * 2 >= distance_trip:
            distance_current = distance_trip / 2
            velocity_current = distance_current / (60 * 60)

            break

        # Ensure current speed never exceeds top speed.
        speed_current = min(speed_current + VEHICLE_DELTA_SPEED_TIME, VEHICLE_TOP_SPEED)
        
    return {
        "distance": distance_current,
        "time": time_elapsed,
        "mph_max": speed_current,
        "velocity_max": velocity_current
    }

### Core Logic: Calculating Tunnel Metrics
Using the the acceleration kinematics as a base, calculate the total time to travel the trip distance.

In [7]:
def calculate_tunnel_metrics(distance_trip: float) -> dict:
    """Calculates metrics for traveling a route via tunnels.
    """
    
    acceleration = simulate_tunnel_vehicle_kinematics(distance_trip)
    
    # Miles
    distance_accelerating = acceleration["distance"]
    distance_decelerating = acceleration["distance"]
    distance_cruising = distance_trip - distance_accelerating - distance_decelerating

    # Seconds
    time_accelerating = acceleration["time"]
    time_decelerating = acceleration["time"]
    time_cruising = distance_cruising / acceleration["velocity_max"]
    time_total = time_accelerating + time_cruising + time_decelerating

    time_total_minutes = time_total / 60
    time_total_hours = time_total_minutes / 60

    speed_max = acceleration["mph_max"]
    speed_avg = distance_trip / time_total_hours

    return {
        "distance": {
            "unit": "mi",
            "value": round(distance_trip, 1),
            "raw": distance_trip
        },
        "time": {
            "unit": "min",
            "value": math.ceil(time_total_minutes), # Round up for a more conservative estimate.
            "raw": time_total_minutes
        },
        "speed_max": {
            "unit": "mph",
            "value": round(speed_max),
            "raw": speed_max
        },
        "speed_avg": {
            "unit": "mph",
            "value": round(speed_avg),
            "raw": speed_avg
        }
    }

### Output UI
Define helper methods to display the Google Map and table of comparison metrics.

In [8]:
def display_map(origin: tuple, destination: tuple):
    route = gmaps_ui.directions_layer(origin, destination)

    map = gmaps_ui.figure()
    map.add_layer(route)
    display(map)

def display_metrics(road: dict, tunnel: dict):
    comparison = f"""| Metric | Road | Tunnel |
    | --- | --- | --- |
    | Distance | {road['distance']['value']} {road['distance']['unit']} | {tunnel['distance']['value']} {tunnel['distance']['unit']} |
    | Top Speed | {road['speed_max']['value']} {road['speed_max']['unit']} | {tunnel['speed_max']['value']} {tunnel['speed_max']['unit']} |
    | Avg Speed | {road['speed_avg']['value']} {road['speed_avg']['unit']} | {tunnel['speed_avg']['value']} {tunnel['speed_avg']['unit']} |
    | **Transit Time** | **{road['time']['value']} {road['time']['unit']}** | **{tunnel['time']['value']} {tunnel['time']['unit']}** |"""

    time_saved = road['time']['value'] - tunnel['time']['value']
    unit = "minutes"
    if time_saved > 60:
        time_saved /= 60
        unit = "hours"
    
    # Render the comparison metrics as a markdown table.
    display(md("### Traditional Roads vs Tunnels Comparison"))   
    display(md(f"**Total Time Saved via Tunnels:** {time_saved} {unit}"))
    display(md("<br>"))
    display(md(comparison))